In [1]:
import sys
# sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")

import gdsfactory as gf
# TODO
# 1. Change one side of the cell_temp to be comparation with the width
# 2. Different with the width and same width with gap

cell_temp = gf.Component()

2025-10-29 09:48:30.758 | INFO     | kfactory.kcell:show:8777 - klive v0.3.3: Opened file '/Users/bubble/Desktop/Project/Piezoelectric/layout/piezoelectrical_sensor/build/gds/1237952483.oas'


In [2]:
# cell
L = [250, 500, 750, 1000]
g = [30, 30, 30, 30, 30]
column_list = [1, 2, 4, 8, 16]
k = len(L) * len(column_list)

w_total = 32
mesh = [None] * k

for i in range(k):
    mesh[i] = gf.Component()
side =[None] * len(column_list)
k = (len(L) * len(g))
mesh = [None] * k
for i in range(k):
    mesh[i] = gf.Component()
for i in range(len(column_list)):
    side[i] = gf.Component()
count = 0
block = gf.Component()
origin = (5000 / len(column_list), 0)

for i in range(len(L)):
    for j in range(len(column_list)):
        w = w_total / column_list[j]
        # single grid
        column = column_list[j]
        height = L[i] - w
        sg = gf.components.rectangle(size=(g[j], height), layer=(8, 0))
        for m in range(column-1):
            sf_ref = mesh[count] << sg
            sf_ref.move((origin[0]+m*(g[j]+w), 0))
        # frame
        fra_width = (column-1) * (g[j] + w) + w
        frame = gf.components.rectangle(size=(fra_width, L[i]), layer=(9, 0))
        frame_ref = mesh[count] << frame
        frame_ref.movex(origin[0]-w)
        # add mesh[count] to side and adjust position
        temp = count // len(column_list)
        mesh_ref = side[temp] << mesh[count]
        mesh_ref.movex((count%(len(column_list))) * 5000 / (len(column_list)+1) - ((column_list[j]-1)*g[j]+32) / 2 - 200) 
        count += 1   
    # length mark
    T = gf.components.text(f"{L[i]}", size=50, layer=(1, 0))
    T_ref = side[i] << T
    T_ref.move((2500, -150))

for i in range(len(side)):
    side_ref = block << side[i]
    if i == 0:
        pass
    elif i == 1:
        side_ref.dmirror_y(5000/2).dmirror_x(5000/2)
    elif i == 2:
        side_ref.drotate(angle=-90, center=(5000/2, 5000/2))
    else:
        side_ref.drotate(angle=90, center=(5000/2, 5000/2))

block.show()

In [4]:
# backside etching 5mm * 5mm (5743.44um)
backside = gf.Component()
size = 5743.44
diff = (size - 5000) / 2
for i in range(4):
    backside_temp = gf.components.rectangle(size=(size, size), layer=(3, 0))
    backside_ref = backside << backside_temp
    if i == 0:
        backside_ref.move((-diff, -diff))
    elif i == 1:
        backside_ref.move((10000-diff, -diff))
    elif i == 2:
        backside_ref.move((-diff, 10000-diff))
    else:
        backside_ref.move((10000-diff, 10000-diff))
# backside.show()

In [5]:
# structure for each die
# structure for 4 sides
fblock = gf.Component()
for i in range(4):
    if i == 0:
        block_ref = fblock << block
    elif i == 1:
        block_ref = fblock << block
        block_ref.move((10000, 0))
    elif i == 2:
        block_ref = fblock << block
        block_ref.move((10000, 10000))
    else:
        block_ref = fblock << block
        block_ref.move((0, 10000))

In [6]:
# order
order = gf.Component()
text_array = []
for i in range(4):
    text_array.append(gf.Component())

for i in range(4):
    T = gf.components.text(f"R1 {i+1}", size=20, layer=(1, 0))
    for j in range(4):
        order_ref = text_array[i] << T
        if j == 0:
            order_ref.move((-60, 0))
        elif j == 1:
            order_ref.move((-60, 5000))
        elif j == 2:
            order_ref.move((5020, 0))
        else:
            order_ref.move((5020, 5000))
    text_array_ref = order << text_array[i]
    if i == 0:
        pass
    elif i == 1:
        text_array_ref.move((10000, 0))
    elif i == 2:
        text_array_ref.move((0, 10000))
    else:
        text_array_ref.move((10000, 10000))





In [7]:
# backside frame 5mm * 5mm 
backframe = gf.Component()
size = 5000
for i in range(4):
    backframe_temp = gf.components.rectangle(size=(size, size), layer=(3, 0))
    backframe_ref = backframe << backframe_temp
    if i == 0:
        backframe_ref.move((0, 0))
    elif i == 1:
        backframe_ref.move((10000, 0))
    elif i == 2:
        backframe_ref.move((0, 10000))
    else:
        backframe_ref.move((10000, 10000))
# backside.show()

In [8]:
# boolean operation
# do backside etching - frame
outside = gf.boolean(A = backframe, B = fblock,  operation="not", layer1=(3, 0), layer2=(9, 0), layer=(2, 0))
outside = gf.boolean(A = outside, B = fblock,  operation="not", layer1=(2, 0), layer2=(10, 0), layer=(1, 0))

cell_temp << outside

# add holes to cell_temp
holes = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(8, 0), layer2=(10, 0), layer=(1, 0))
cell_temp << holes
# add holes to cell_temp
pc = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(11, 0), layer2=(30, 0), layer=(2, 0))
cell_temp << pc
cell_temp << backside
# add marker/order
marker = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(1, 0), layer2=(10, 0), layer=(1, 0))
cell_temp << marker
cell_temp << order
# frame
frame1 = gf.components.rectangle(size=(15000, 15000), layer=(20, 0))
frame2 = gf.components.rectangle(size=(20000, 20000), layer=(21, 0))
frame2_ref = cell_temp << frame2
frame2_ref.move((-2500, -2500))
cell_temp << frame1

cell_resonator = gf.Component()
cell_ref = cell_resonator << cell_temp
cell_ref.move((2500, -7500))
# cell_resonator.show()
cell_resonator.write_gds("cell_resonator.gds")
# cell_resonator.plot()

PosixPath('cell_resonator.gds')